## Caso de uso - Calculo del saldo comercial

Que significa saldo comercial

In [1]:
# importacion de pyspark
import pyspark

In [2]:
# identificacion del spark context
sc

<SparkContext master=local[*] appName=PySparkShell>

## Importacion de datasets

### Importacion del clasificador CIIU

In [26]:
# importacion del clasificador CIIU 
# el archivo fisico debe estar cargado en el HDFS (/user/hadoop/....)
dataset_ciiu = spark.read.option("header",True).csv("ciiur3_post3.csv")

In [27]:
dataset_ciiu.show()

+------+--------------------+-----+
|codigo|         descripcion|nivel|
+------+--------------------+-----+
|     A|Agricultura, gana...|    1|
|    01|Agricultura, gana...|    2|
|   011|Cultivos en gener...|    3|
|  0111|Cultivo de cereal...|    4|
|  0112|Cultivo de hortal...|    4|
|  0113|Cultivo de frutas...|    4|
|   012|   Cría de animales.|    3|
|  0121|Cría de ganado va...|    4|
|  0122|Cría de otros ani...|    4|
|   013|Cultivo de produc...|    3|
|  0130|Cultivo de produc...|    4|
|   014|Actividades de se...|    3|
|  0140|Actividades de se...|    4|
|   015|Caza ordinaria y ...|    3|
|  0150|Caza ordinaria y ...|    4|
|    02|Silvicultura, ext...|    2|
|   020|Silvicultura, ext...|    3|
|  0200|Silvicultura, ext...|    4|
|     B|              Pesca.|    1|
|    05|Pesca, explotació...|    2|
+------+--------------------+-----+
only showing top 20 rows



In [28]:
# temporal para ciiu
dataset_ciiu.createOrReplaceTempView("ciiu")

### Importacion del dataset EXPORTACIONES

In [29]:
# importacion del dataset export
# el archivo fisico debe estar cargado en el HDFS (/user/hadoop/....)

dataset_export = spark.read\
    .option("inferSchema","true")\
    .option("header",True)\
    .csv("export_post3.csv")

In [30]:
# muestra del dataset
dataset_export.show(10)

+-------+---+------------+------+---------+----------+----------+----------+
|gestion|mes|departamento|ciiur3|    valor|    kilbru|    kilnet|   nandina|
+-------+---+------------+------+---------+----------+----------+----------+
|   2019|  5|  SANTA CRUZ|  1514|1689360.0| 3000000.0| 3000000.0|1507100000|
|   2018|  4|  CHUQUISACA|  1110|511854.68|2017313.27|2017313.27|2711210000|
|   2018|  7|  SANTA CRUZ|  1911| 84828.68|     341.5|     341.5|4113300000|
|   2018|  5|      LA PAZ|  1920|  1414.05|     585.0|     580.0|6402200000|
|   2018|  9|  COCHABAMBA|  3694|  3574.14|     51.13|     46.93|9503002200|
|   2018|  7|       PANDO|  0113| 147730.0|  16686.72|  15966.72| 801220000|
|   2018|  7|        BENI|  0113|1942793.6|  218160.0|  208000.0| 801220000|
|   2018|  7|        BENI|  0113| 847783.2|   93281.0|   89000.0| 801220000|
|   2018|  7|       PANDO|  0113| 123728.0|  16686.72|  15966.72| 801220000|
|   2018|  7|        BENI|  0113| 237260.0|   42000.0|   40000.0| 801220000|

In [31]:
# esquema del dataset
dataset_export.printSchema()

root
 |-- gestion: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- departamento: string (nullable = true)
 |-- ciiur3: string (nullable = true)
 |-- valor: double (nullable = true)
 |-- kilbru: double (nullable = true)
 |-- kilnet: double (nullable = true)
 |-- nandina: long (nullable = true)



In [32]:
# creacion de una vista temporal para exportacion
dataset_export.createOrReplaceTempView("exportaciones")

In [33]:
# agrupamos por gestion y ciiu y el valor en millones
exportaciones_agrupado = spark.sql("""
SELECT gestion, ciiur3, departamento, sum(valor)/1000000 AS valor_millones
FROM exportaciones
GROUP BY gestion, ciiur3,departamento
order by gestion, departamento, ciiur3 asc
""")

In [34]:
# agrupamos exportaciones por gestion ciiu y departamento
exportaciones_agrupado.createOrReplaceTempView("exportaciones_agrupado")

In [35]:
# muestra del dataset
exportaciones_agrupado.show(10)

+-------+------+------------+------------------+
|gestion|ciiur3|departamento|    valor_millones|
+-------+------+------------+------------------+
|   2016|  0111|        BENI|         0.0133175|
|   2016|  0113|        BENI|154.10162598000002|
|   2016|  1320|        BENI|2.7991595700000005|
|   2016|  1511|        BENI|              0.12|
|   2016|  1514|        BENI|          2.992E-4|
|   2016|  1911|        BENI|            0.0252|
|   2016|  2010|        BENI|2.3657291999999996|
|   2016|  2720|        BENI|      179.57699444|
|   2016|  2911|        BENI|              0.01|
|   2016|  0111|  CHUQUISACA|2.1779320099999997|
+-------+------+------------+------------------+
only showing top 10 rows



### Importacion del dataset de IMPORTACIONES

In [36]:
# importacion del dataset import
# el archivo fisico debe estar cargado en el HDFS (/user/hadoop/....)

dataset_import = spark\
    .read\
    .option("inferSchema",True)\
    .option("header",True)\
    .csv("import_post3.csv")

In [38]:
# muestra del dataset import
dataset_import.show(10)

+-------+---+-------+-------+-------+------+----------+------------+
|gestion|mes|  kilos|    fro|    fob|ciiur3|   nandina|departamento|
+-------+---+-------+-------+-------+------+----------+------------+
|   2020|  4|12600.0|13134.0|11100.0|  3410|8408209000|      LA PAZ|
|   2020|  4|  300.0|  320.0|  270.0|  3430|8708302900|      LA PAZ|
|   2020|  6|  284.0| 8017.0| 7872.0|  3430|8708992900|  SANTA CRUZ|
|   2020|  4|   10.0|   26.0|   25.0|  3430|8708299000|      LA PAZ|
|   2020|  4|13934.0|28933.0|27262.0|  3430|8708299000|      LA PAZ|
|   2020|  4| 1137.0| 1051.0|  925.0|  3430|8708302900|      LA PAZ|
|   2020|  4|  500.0|  598.0|  540.0|  3430|8708401000|      LA PAZ|
|   2020|  4| 2208.0| 4725.0| 4260.0|  3430|8708501100|      LA PAZ|
|   2020|  4| 6686.0|17714.0|16745.0|  3430|8708501900|      LA PAZ|
|   2020|  4| 1382.0| 2665.0| 2380.0|  3430|8708100000|      LA PAZ|
+-------+---+-------+-------+-------+------+----------+------------+
only showing top 10 rows



In [39]:
# esquema del dataset
dataset_import.printSchema()

root
 |-- gestion: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- kilos: double (nullable = true)
 |-- fro: double (nullable = true)
 |-- fob: double (nullable = true)
 |-- ciiur3: string (nullable = true)
 |-- nandina: long (nullable = true)
 |-- departamento: string (nullable = true)



In [40]:
# creando temporal de import
dataset_import.createOrReplaceTempView("importaciones")

In [41]:
importaciones_agrupado = spark.sql("""
SELECT gestion, departamento, ciiur3, sum(fob)/1000000 AS fob_millones
FROM importaciones
GROUP BY gestion, departamento,ciiur3
order by gestion, departamento,ciiur3 asc
""")

In [42]:
# muesta de importaciones agrupado
importaciones_agrupado.show(10)

+-------+------------+------+------------+
|gestion|departamento|ciiur3|fob_millones|
+-------+------------+------+------------+
|   2016|        BENI|  0111|    0.002396|
|   2016|        BENI|  0113|    0.843606|
|   2016|        BENI|  1410|    0.105641|
|   2016|        BENI|  1422|      1.4E-5|
|   2016|        BENI|  1429|     0.00119|
|   2016|        BENI|  1511|    1.467525|
|   2016|        BENI|  1513|    0.166014|
|   2016|        BENI|  1514|    0.418447|
|   2016|        BENI|  1520|    0.027361|
|   2016|        BENI|  1531|    0.159783|
+-------+------------+------+------------+
only showing top 10 rows



In [43]:
# creamos vista temporal de importaciones agrupado
importaciones_agrupado.createOrReplaceTempView("importaciones_agrupado")

In [62]:
# Calculo final del saldo comercial
# el saldo comercial es la diferencia entre importaciones y exportaciones
# el saldo comercial negativo indica que se importa mas de lo que se exporta de productos de una determinada actividad
# los saldos comerciales negativos mas grandes son candidatos a un nicho de mercado
saldo_comercial = spark.sql("""
SELECT i.gestion, i.departamento,ciiu.descripcion, round(i.fob_millones,3) import_millones, round(e.valor_millones, 3) export_millones,
round(e.valor_millones - i.fob_millones, 3) saldo, IF(round(e.valor_millones - i.fob_millones, 3) < 0, 1, 0) outcome 
FROM importaciones_agrupado i
inner join ciiu on ciiu.codigo = i.ciiur3
inner join exportaciones_agrupado e on (e.gestion = i.gestion and e.departamento = i.departamento and i.ciiur3 = e.ciiur3)
order by gestion, departamento, saldo desc
""")

In [63]:
saldo_comercial.show(20)

+-------+------------+--------------------+---------------+---------------+-------+-------+
|gestion|departamento|         descripcion|import_millones|export_millones|  saldo|outcome|
+-------+------------+--------------------+---------------+---------------+-------+-------+
|   2016|        BENI|Fabricación de pr...|            0.0|        179.577|179.577|      0|
|   2016|        BENI|Cultivo de frutas...|          0.844|        154.102|153.258|      0|
|   2016|        BENI|Aserrado y acepil...|           0.38|          2.366|  1.985|      0|
|   2016|        BENI|Cultivo de cereal...|          0.002|          0.013|  0.011|      0|
|   2016|        BENI|Fabricación de mo...|          0.012|           0.01| -0.002|      1|
|   2016|        BENI|Elaboración de ac...|          0.418|            0.0| -0.418|      1|
|   2016|        BENI|Producción, proce...|          1.468|           0.12| -1.348|      1|
|   2016|  CHUQUISACA|Fabricación de pr...|          0.054|           6.59|  6.5

In [64]:
# guardando el saldo comercial nacional
saldo_comercial.write.option("header",True).csv("saldo_comercial_nacional.csv")

In [61]:
# guardando el dataset en formato avro
#saldo_comercial.write.format("avro").save("saldo_comercial.avro")

In [52]:
saldo_comercial.createOrReplaceTempView("saldo_comercial_nacional");

In [56]:
# analizando el saldo comercial de La Paz en la gestion 2020
saldo_comercial_departamento = spark.sql("""
SELECT gestion,  descripcion ,saldo
FROM saldo_comercial_nacional
WHERE departamento = 'LA PAZ'
and gestion = 2020
ORDER BY gestion, saldo asc
""")

In [57]:
saldo_comercial_departamento.show(100, False)

+-------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|gestion|descripcion                                                                                                                                         |saldo   |
+-------+----------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|2020   |Fabricación de vehículos automotores.                                                                                                               |-144.404|
|2020   |Fabricación de productos de la refinación del petróleo.                                                                                             |-119.022|
|2020   |Fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos.                                                      |-7

###  Conclusiones



##### Segun el saldo comercial negativo podemos observar que productos en las siguientes actividades son candidatas a ser nichos de mercado factibles
| Gestion      | Actividad | Saldo negativo|
| ----------- | ----------- |---------------|
    |2020    |Fabricación de vehículos automotores.|-144.404|
    |2020   |Fabricación de productos de la refinación del petróleo.|-119.022|
    |2020   |Fabricación de productos farmacéuticos, sustancias químicas medicinales y productos botánicos.|-78.38|
    |2020   |Fabricación de productos de plástico.|-65.432 |
    |2020   |Fabricación de equipo médico y quirúrgico y de aparatos ortopédicos. |-64.844 |
    |2020   |Fabricación de maquinaria para la explotación de minas y canteras y para obras de construcción.|-59.586| 
